In [4]:
import polars as pl

In [122]:
qrq_data = {
    x: pl.read_csv(
        f'../data/historic-data/qrq-{x}.csv',
        null_values = 'NA'
    )
    for x in ['raw', 'clean']
}

roli_data = (
    pl.read_excel('../data/historic-data/ROLI-data.xlsx')
    .unpivot(index = ['country', 'year', 'code', 'region'])
    .filter(pl.col('year').is_in(['2019', '2020', '2021', '2022', '2023', '2024']))
    .with_columns(
        pl.col('year').cast(pl.Int64())
    )
    .select(['country', 'year', 'variable', 'value'])
)

country_additions = {
    '2020': ['The Gambia', 'Kosovo'],
    '2021': ['Congo, Rep.', 'Cyprus', 'Haiti', 'Ireland', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Paraguay', 'Slovak Republic', 'Sudan'],
    '2022': ['Gabon'],
    '2023': ['Kuwait', 'Montenegro']
}

country_additions_df = pl.DataFrame({
    "edition": [year for year, countries in country_additions.items() for _ in countries],
    "country": [country for countries in country_additions.values() for country in countries]
}).with_columns(
    pl.col('edition').cast(pl.Int64())
)

name_corrections = {
    "Bahamas"           : "The Bahamas",
    "Cote d'Ivoire"     : "Côte d'Ivoire",
    "Czech Republic"    : "Czechia",
    "Egypt"             : "Egypt, Arab Rep.",
    "Gambia"            : "The Gambia",
    "Iran"              : "Iran, Islamic Rep.",
    "Kyrgyzstan"        : "Kyrgyz Republic",
    "Macedonia, FYR"    : "North Macedonia",
    "Republic of Korea" : "Korea, Rep.",
    "Russia"            : "Russian Federation",
    "Turkey"            : "Türkiye",
    "Turkiye"           : "Türkiye",
    "Venezuela"         : "Venezuela, RB"
}

The ML model is meant to capture the following relationship:

$
flag_{i,t} = f(d2m_{i,t}, d2lm_{i,t}, d2pym_{i,t}, looi_{i,t}, n_{g,t}, l_{i,t}, y2e_{i,t}, tps_{i,t})
$

where,

- $flag = \begin{cases} 1 &\text{if individual was dropped from the final sample} \\ 0 &\text{if individual is present in the final sample} \end{cases}$

- $d2m = s_n - s_i$, and $s_n$ is te average group $g$ score and $s_i$ is the score obtained by the individual

- $d2lm = \dfrac{s_l - s_i}{d2m}$, and $s_l$ is the average longitudinal group $g$ score

- $d2pym = \dfrac{s_py - s_i}{d2m}$, and $s_py$ is the average group $g$ score for $t-1$

- $looi = s_n - s_loo$, and $s_loo$ is the leave-one-out average score for a given individual

- $n$ is the size of group $g$

- $l = \begin{cases} 1*lp_{g,t} &\text{if individual is longitudinal} \\ 0 &\text{if individual is not longitudinal} \end{cases}$, and $lp$ is the proportion of longitudinal experts in group $g$

- $y2e$ is the number of years between the individual's answers and the ROLI edition

- $tps$ ... TBD

In [ ]:
qrq_data_raw_processed = (
    qrq_data['raw']
    .join(
        country_additions_df,
        on = ['country', 'edition'],
        how = 'anti'
    )
    .with_columns(
        (pl.col('unid').is_in(qrq_data['clean']['unid']).not_())
        .cast(pl.Int8)
        .alias('dropped'),
        
        pl.col('roli').mean().over('country', 'edition').alias('avg_country_score'),
        pl.col('roli').std().over('country', 'edition').alias('std_country_score'),

        pl.col('roli').sum().over('country', 'edition').alias('sum_roli'),
        pl.col('roli').count().over('country', 'edition').alias('count_roli'),

        pl.col('roli')
        .filter(pl.col('longitudinal') == 1)
        .mean()
        .over('country', 'edition')
        .alias('avg_country_longitudinal_score'),

        pl.len().over('country', 'edition').alias('pool_size'),

        pl.len().over('country', 'edition', 'question').alias('question_pool'),

        pl.col('longitudinal').sum().over('country', 'edition').alias('longitudinal_pool_size'),

        (pl.col('edition') - pl.col('year') )
        .alias('distance2edition'),

        pl.col('country').replace_strict(
            name_corrections,
            default = pl.col('country')
        ).alias('country')
    )
    .with_columns(
        (pl.col('roli') - pl.col('avg_country_score'))
        .alias('distance2mean'),
        
        (pl.col('longitudinal') * (pl.col('longitudinal_pool_size') / pl.col('pool_size'))).alias('longitudinal_prop'),

        (pl.col('avg_country_score') - ((pl.col('sum_roli') - pl.col('roli')) / (pl.col('count_roli') - 1))).alias('leave_one_out_score')
    )
    .join(
        (
            roli_data
            .filter(pl.col('variable') == 'roli')
            .select(['country', 'year', 'value'])
            .with_columns(
                (pl.col('year') + 1)
            )
            .rename({'value' : 'prev_year_avg_score'})
        ),
        left_on  = ['country', 'edition'],
        right_on = ['country', 'year'],
        how = 'left'
    )
    .with_columns(
        (pl.col('distance2mean') / pl.col('std_country_score')).alias('distance2mean_nstd'),
        ((pl.col('avg_country_longitudinal_score') - pl.col('roli'))/(1+pl.col('distance2mean'))).alias('distance2long'),
        ((pl.col('prev_year_avg_score') - pl.col('roli'))/(1+pl.col('distance2mean'))).alias('distance2pyear')
    )
)

In [126]:
features = (
    qrq_data_raw_processed
    .select(['country', 'edition', 'dropped', 'distance2mean', 'distance2long', 'distance2pyear', 
             'leave_one_out_score', 'question_pool', 'longitudinal_prop', 'distance2edition'])
)

In [127]:
x = features.filter(pl.col('distance2long').is_null())